In [10]:
import numpy as np
import keras
from sklearn.model_selection import train_test_split

# Load data
images = np.load('images_75.npy')
labels = np.load('labels_75.npy')

# Normalize images to [0, 1] and add channel dimension
images = images.astype('float32') / 255.0
images = np.expand_dims(images, -1) 

print("Images shape:", images.shape)

# split into train, val and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(images, labels, test_size=0.2, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)


Images shape: (18000, 75, 75, 1)


Task 2.1 Classification

Creating 24 categories (30 minute interval)

In [4]:
def to_class_label(hours, minutes, interval=30):
    # into 30 minute intervals (interval 0 = 00:00-00:29)
    total_minutes = (hours % 12) * 60 + minutes
    return total_minutes // interval  

# Create class labels for 24 categories (30 minute intervals)
y_train_24 = to_class_label(y_train[:,0], y_train[:,1])
y_val_24 = to_class_label(y_val[:,0], y_val[:,1])
y_test_24 = to_class_label(y_test[:,0], y_test[:,1])

from tensorflow.keras.utils import to_categorical
y_train_24 = to_categorical(y_train_24, 24)
y_val_24 = to_categorical(y_val_24, 24)
y_test_24 = to_categorical(y_test_24, 24)

2025-10-28 20:36:14.216291: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 20:36:14.229854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761680174.244322 1817126 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761680174.247866 1817126 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761680174.258504 1817126 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
def common_sense_err(y_true, y_pred):
    #todo adapt later to compare the original exact time values
    # Convert one-hot encoded predictions to time indices
    y_pred_classes = tf.argmax(y_pred, axis=1)
    y_true_classes = tf.argmax(y_true, axis=1)
    
    # Convert to minutes (multiply by 30 since each class represents 30 minutes)
    pred_minutes = tf.cast(y_pred_classes * 30, tf.float32)
    true_minutes = tf.cast(y_true_classes * 30, tf.float32)
    
    # Calculate absolute difference
    diff = tf.abs(pred_minutes - true_minutes)
    
    # Apply circular correction (if diff > 360, take 720 - diff)
    circular_diff = tf.minimum(diff, 720 - diff)
    
    # Return mean difference in minutes
    return tf.reduce_mean(circular_diff)

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

def time_classifier(X_train, y_train, X_val, y_val, pixel_size=75, num_categories=24):
    model = models.Sequential([
        layers.Input(shape=(pixel_size, pixel_size, 1)),
        layers.Conv2D(32, (3, 3),padding = 'same', activation='relu'),#, input_shape=(pixel_size, pixel_size, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Conv2D(64, (3, 3), padding = 'same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Conv2D(128, (3, 3),padding = 'same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)), 
        
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_categories, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))

In [16]:
# ...existing code...
y_train_24 = y_train_24.astype('float32')
y_val_24   = y_val_24.astype('float32')
y_test_24  = y_test_24.astype('float32')
print("y_train_24 dtype:", y_train_24.dtype)
# ...existing code...

y_train_24 dtype: float32


In [17]:
time_classifier(X_train, y_train_24, X_val, y_val_24, pixel_size=75, num_categories=24)

2025-10-28 20:45:09.838906: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 308.99MiB (rounded to 324000000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-10-28 20:45:09.838930: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-10-28 20:45:09.838938: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 67, Chunks in use: 67. 16.8KiB allocated for chunks. 16.8KiB in use in bin. 3.2KiB client-requested in use in bin.
2025-10-28 20:45:09.838941: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 10, Chunks in use: 10. 5.2KiB allocated for chunks. 5.2KiB in use in bin. 4.8KiB client-requested in use in bin.
2025

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [13]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
optimizer="sgd",
metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=30,validation_data=(X_val, y_val))

/vol/home/s4557093/.conda/envs/tf-gpu/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2025-10-28 20:43:36.232196: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 308.99MiB (rounded to 324000000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-10-28 20:43:36.232218: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-10-28 20:43:36.232223: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 36, Chunks in use: 36. 9.0KiB allocated for chunks. 9.0KiB in use in bin. 720B client-requested in use in bin.
2025-10-28 20:43:36.232225: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 3, Chunks in use: 3. 1.5KiB allocated for chunks. 1.5KiB in use in bin. 1.3KiB client-requested in use in bin.
2025-10-28

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [14]:
import inspect, numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers

# 2) quick build of a simple test model (no BN/Dropout) and print summary
def quick_model(X_train, y_train, X_val, y_val, pixel_size=75, num_categories=24):
    m = models.Sequential([
        layers.Input(shape=(pixel_size,pixel_size,1)),
        layers.Conv2D(32,3,padding='same',activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64,3,padding='same',activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128,activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_categories,activation='softmax')
    ])
    m.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    history = m.fit(X_train, y_train, epochs=150, batch_size=64, validation_data=(X_val, y_val))

#quick_model(X_train[:100], y_train_24[:100], X_val, y_val_24, pixel_size=75, num_categories=24)
quick_model(X_train, y_train_24, X_val, y_val_24, pixel_size=75, num_categories=24)

2025-10-28 20:44:09.052046: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 308.99MiB (rounded to 324000000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-10-28 20:44:09.052067: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-10-28 20:44:09.052075: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 43, Chunks in use: 43. 10.8KiB allocated for chunks. 10.8KiB in use in bin. 1.2KiB client-requested in use in bin.
2025-10-28 20:44:09.052079: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 4, Chunks in use: 4. 2.0KiB allocated for chunks. 2.0KiB in use in bin. 1.8KiB client-requested in use in bin.
2025-1

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

38
2025-10-28 20:44:09.052185: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 7fada0001f00 of size 256 next 36
2025-10-28 20:44:09.052188: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 7fada0002000 of size 256 next 43
2025-10-28 20:44:09.052189: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 7fada0002100 of size 256 next 41
2025-10-28 20:44:09.052191: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 7fada0002200 of size 256 next 42
2025-10-28 20:44:09.052192: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 7fada0002300 of size 256 next 46
2025-10-28 20:44:09.052194: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 7fada0002400 of size 256 next 47
2025-10-28 20:44:09.052196: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1114] InUse at 7fada0002500 of size 256 next 48
2025-10-28 20:44:09.052197: I external/local_xla/xla/tsl/framework/bfc_al